# Keras基本例からNumerai


In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

Using TensorFlow backend.


## 使用データ

同じフォルダにnumeraiのtrainingデータセットおくこと.

In [2]:
inputCSV = pd.read_csv('numerai_training_data.csv')
inputCSV.head(10)

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,target
0,0.592482,0.406464,0.130613,0.585993,0.203948,0.990577,0.065272,0.864658,0.850340,0.116827,...,0.976095,0.523960,0.148403,0.600202,0.688142,0.470819,0.286410,0.796651,0.641891,1
1,0.097404,0.135520,0.120604,0.077005,0.564318,0.066696,0.675741,0.375746,0.927066,0.091658,...,0.081815,0.128766,0.076859,0.839097,0.576018,0.759510,0.745820,0.303073,0.878545,0
2,0.910402,0.941407,0.209146,0.912570,0.785759,0.877535,0.762701,0.643769,0.108549,0.293277,...,0.943610,0.822201,0.333061,0.782492,0.666455,0.007921,0.608245,0.652344,0.033444,1
3,0.938794,0.980285,0.452220,0.896527,0.909025,0.391896,0.741063,0.801898,0.133652,0.573551,...,0.930512,0.853767,0.317888,0.460311,0.063970,0.434131,0.202171,0.906957,0.297918,1
4,0.240828,0.754414,0.888972,0.228895,0.172946,0.593818,0.180354,0.623804,0.348972,0.715115,...,0.671845,0.324413,0.856480,0.276030,0.231954,0.333042,0.517633,0.269451,0.320914,1
5,0.250720,0.402564,0.742476,0.332391,0.586591,0.719378,0.612262,0.709459,0.487905,0.938784,...,0.347437,0.016488,0.746829,0.393725,0.658273,0.969089,0.552002,0.707005,0.762388,1
6,0.301617,0.020043,0.918485,0.445900,0.976656,0.035959,0.994657,0.112611,0.691903,0.823239,...,0.008203,0.296781,0.959141,0.976805,0.592507,0.604772,0.538080,0.000875,0.611522,0
7,0.418199,0.767178,0.065719,0.423229,0.624257,0.799378,0.572715,0.767760,0.619077,0.191164,...,0.685799,0.517264,0.178209,0.595161,0.841767,0.486965,0.813587,0.658656,0.646261,1
8,0.303557,0.328762,0.829169,0.212381,0.988171,0.146131,0.885027,0.185728,0.188507,0.981038,...,0.384778,0.327223,0.983245,0.924070,0.080164,0.333042,0.116667,0.206736,0.259746,0
9,0.299323,0.634118,0.094413,0.488458,0.636429,0.557253,0.248635,0.981777,0.872428,0.002076,...,0.569149,0.438558,0.007408,0.282386,0.395210,0.723274,0.222967,0.853229,0.782230,0


In [3]:
# データの並びをシャッフル
calcData = inputCSV.reindex(np.random.permutation(inputCSV.index))
calcData.head(10)

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,target
48473,0.267237,0.264007,0.116871,0.000138,0.928022,0.068729,0.628379,0.559896,0.971765,0.610268,...,0.151081,0.371125,0.528316,0.911122,0.360028,0.367189,0.568665,0.368693,0.457101,0
34953,0.289866,0.861914,0.249400,0.444875,0.181340,0.858578,0.104368,0.587745,0.549819,0.715115,...,0.713052,0.410386,0.636224,0.260473,0.599239,0.160721,0.452185,0.468886,0.237294,0
60858,0.244921,0.549913,0.600992,0.489465,0.239297,0.462402,0.018159,0.612390,0.336934,0.852291,...,0.261007,0.354061,0.771981,0.003988,0.019908,0.265608,0.099969,0.437955,0.433236,0
37041,0.274483,0.111568,0.044299,0.233717,0.399857,0.217382,0.341818,0.959074,0.787275,0.129955,...,0.179476,0.218583,0.044701,0.339477,0.234977,0.353886,0.160570,0.945028,0.715157,1
14422,0.563806,0.085910,0.354630,0.425055,0.711377,0.411280,0.742378,0.368013,0.322918,0.290961,...,0.201743,0.392445,0.662487,0.777398,0.308180,0.585882,0.574256,0.371864,0.451790,0
32729,0.808552,0.937504,0.577363,0.725515,0.665610,0.982281,0.369034,0.570566,0.204822,0.442739,...,0.809670,0.788433,0.576929,0.534686,0.163943,0.225236,0.206333,0.554527,0.233900,1
85191,0.283136,0.217313,0.257578,0.208455,0.803595,0.237082,0.728964,0.233684,0.983227,0.071147,...,0.266642,0.099063,0.210823,0.570068,0.621894,0.377919,0.768661,0.360165,0.409194,0
78786,0.657740,0.812972,0.398299,0.747423,0.642095,0.715388,0.349160,0.229968,0.505458,0.390660,...,0.685799,0.810836,0.499223,0.338154,0.355965,0.358662,0.298091,0.462977,0.473590,1
24399,0.181035,0.703914,0.184222,0.212269,0.715664,0.257961,0.821313,0.141390,0.818261,0.056567,...,0.159951,0.595791,0.149229,0.615686,0.255971,0.745703,0.585825,0.418035,0.950710,0
77574,0.206047,0.306965,0.344082,0.348124,0.633733,0.647077,0.687997,0.786589,0.766099,0.299951,...,0.391576,0.191053,0.353461,0.669273,0.308180,0.697753,0.528111,0.487878,0.682311,0


In [4]:
# 入力データと教師データに分割
pd_y = calcData['target']
pd_x = calcData.drop(['target'], axis=1)

# numpy配列にデータフレームから変換
x_array = pd_x.as_matrix()
y_array = pd_y.as_matrix()

In [7]:
pd_x.shape[0]

96320

In [13]:
# 正規化し,float32の型に変換
x_scale = scale(x_array).astype(np.float32)

# サンプルの一部をテストデータ用に分割
N = pd_x.shape[0] * 0.1
X_train, X_test = np.split(x_scale, [N])

/Users/kohno/.pyenv/versions/3.5.1/envs/devkeras/lib/python3.5/site-packages/numpy/lib/shape_base.py:422: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  sub_arys.append(_nx.swapaxes(sary[st:end], axis, 0))


### 教師データのラベリング
変換が必要な点に注意.

In [29]:
batch_size = 2000
nb_classes = 2 # 2値分類
nb_epoch = 200

In [30]:
y_scale = y_array.astype(np.int32)
Y_train, Y_test = np.split(y_scale, [N])

# 離散の分類問題の場合、ベクトルのラベルをto_categoricalで変換しておく必要がある.
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(Y_train, nb_classes)
Y_test = np_utils.to_categorical(Y_test, nb_classes)

/Users/kohno/.pyenv/versions/3.5.1/envs/devkeras/lib/python3.5/site-packages/numpy/lib/shape_base.py:422: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  sub_arys.append(_nx.swapaxes(sary[st:end], axis, 0))


## Deep learningのモデル定義

以下では暫定的に多層ニューラルネットを用いる.

(ここが基本的に変更する場所である.)

なお、活性化関数にReLUを用いる.

また途中に過学習を防ぎ汎化性能を上げる手法であるdropoutが仕込んである.

In [35]:
model = Sequential()
model.add(Dense(100, input_shape=(21,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(2))
model.add(Activation('sigmoid'))

### モデルは一度コンパイルする形式を取るタイプ

サマリーが出てちょっと見やすい.

In [36]:
model.summary()

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_22 (Dense)                 (None, 100)           2200        dense_input_7[0][0]              
____________________________________________________________________________________________________
activation_22 (Activation)       (None, 100)           0           dense_22[0][0]                   
____________________________________________________________________________________________________
dropout_16 (Dropout)             (None, 100)           0           activation_22[0][0]              
____________________________________________________________________________________________________
dense_23 (Dense)                 (None, 100)           10100       dropout_16[0][0]                 
___________________________________________________________________________________________

### 学習と結果

fitで学習し,evaluateでtestデータから汎化性能をチェックする.

In [37]:
history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 9632 samples, validate on 86688 samples
Epoch 1/200
9632/9632 [==============================] - 0s - loss: 0.7061 - acc: 0.4918 - val_loss: 0.6942 - val_acc: 0.5064
Epoch 2/200
9632/9632 [==============================] - 0s - loss: 0.6970 - acc: 0.5113 - val_loss: 0.6940 - val_acc: 0.5026
Epoch 3/200
9632/9632 [==============================] - 0s - loss: 0.6968 - acc: 0.5062 - val_loss: 0.6931 - val_acc: 0.5123
Epoch 4/200
9632/9632 [==============================] - 0s - loss: 0.6948 - acc: 0.5110 - val_loss: 0.6935 - val_acc: 0.5049
Epoch 5/200
9632/9632 [==============================] - 0s - loss: 0.6956 - acc: 0.5065 - val_loss: 0.6929 - val_acc: 0.5134
Epoch 6/200
9632/9632 [==============================] - 0s - loss: 0.6941 - acc: 0.5130 - val_loss: 0.6928 - val_acc: 0.5143
Epoch 7/200
9632/9632 [==============================] - 0s - loss: 0.6929 - acc: 0.5206 - val_loss: 0.6929 - val_acc: 0.5125
Epoch 8/200
9632/9632 [==============================] - 0s - loss: 0

In [38]:
pred = pd.read_csv("numerai_tournament_data.csv")
pred.head(10)

,t_id,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21
0,35127,0.093420,0.296856,0.416888,0.271249,0.609466,0.272623,0.455763,0.719717,0.594638,...,0.612889,0.129737,0.015982,0.717095,0.396762,0.572472,0.942516,0.549933,0.875827,0.866483
1,5815,0.945969,0.520152,0.913747,0.947708,0.876677,0.183711,0.994657,0.091799,0.176480,...,0.122915,0.314198,0.845424,0.932939,0.932285,0.009922,0.040073,0.240286,0.252535,0.196575
2,21613,0.375753,0.131812,0.398299,0.714723,0.038811,0.286954,0.016110,0.159726,0.176476,...,0.130703,0.088841,0.475284,0.145227,0.035899,0.056418,0.410543,0.061295,0.066628,0.675992
3,24162,0.033064,0.131812,0.032016,0.141477,0.454620,0.462402,0.168206,0.171101,0.931317,...,0.707663,0.169385,0.340110,0.235432,0.061764,0.758716,0.843956,0.275956,0.070583,0.919119
4,13935,0.142782,0.071785,0.468688,0.408943,0.911702,0.101406,0.909998,0.223743,0.517424,...,0.967458,0.024189,0.060593,0.130316,0.834466,0.958453,0.933132,0.954030,0.143196,0.811200
5,6769,0.664717,0.826183,0.276454,0.283183,0.119841,0.997214,0.424611,0.547748,0.851347,...,0.713724,0.882243,0.595791,0.421879,0.461548,0.371382,0.335522,0.530886,0.391486,0.361026
6,23101,0.584238,0.761102,0.724517,0.697422,0.576719,0.687706,0.647262,0.159726,0.389062,...,0.612459,0.718174,0.579461,0.806217,0.562815,0.625256,0.472099,0.670752,0.109898,0.585839
7,27270,0.167520,0.051942,0.331846,0.173939,0.819964,0.379880,0.512372,0.761694,0.594569,...,0.207911,0.334589,0.064694,0.389487,0.399285,0.124956,0.755826,0.075363,0.438344,0.626302
8,2044,0.192413,0.229133,0.309713,0.173912,0.801884,0.259285,0.844189,0.143807,0.922540,...,0.572723,0.084591,0.372183,0.184446,0.441292,0.740264,0.200827,0.379106,0.135458,0.525279
9,10270,0.305077,0.526421,0.945373,0.585993,0.946542,0.273140,0.798628,0.207506,0.044341,...,0.919853,0.322999,0.343980,0.436126,0.929317,0.940058,0.033285,0.952063,0.009844,0.617795


In [42]:
pred_x = pred.drop(["t_id"],axis=1)
pred_x.head(3)

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21
0,0.093420,0.296856,0.416888,0.271249,0.609466,0.272623,0.455763,0.719717,0.594638,0.916434,...,0.612889,0.129737,0.015982,0.717095,0.396762,0.572472,0.942516,0.549933,0.875827,0.866483
1,0.945969,0.520152,0.913747,0.947708,0.876677,0.183711,0.994657,0.091799,0.176480,0.938784,...,0.122915,0.314198,0.845424,0.932939,0.932285,0.009922,0.040073,0.240286,0.252535,0.196575
2,0.375753,0.131812,0.398299,0.714723,0.038811,0.286954,0.016110,0.159726,0.176476,0.125447,...,0.130703,0.088841,0.475284,0.145227,0.035899,0.056418,0.410543,0.061295,0.066628,0.675992


In [44]:
np_pred = pred_x.as_matrix()

ans = model.predict(np_pred)

In [50]:
pd_ans = pd.DataFrame(ans)
pd_ans[1]

0        0.312819
1        0.534440
2        0.645487
3        0.470157
4        0.397954
5        0.432818
6        0.547083
7        0.486919
8        0.485804
9        0.549066
10       0.519676
11       0.374994
12       0.552595
13       0.432911
14       0.487856
15       0.472630
16       0.586581
17       0.637877
18       0.446020
19       0.563931
20       0.492324
21       0.522402
22       0.428217
23       0.469244
24       0.455476
25       0.529906
26       0.587196
27       0.523455
28       0.287439
29       0.544257
           ...   
36042    0.418537
36043    0.322587
36044    0.430763
36045    0.377407
36046    0.450796
36047    0.409155
36048    0.554193
36049    0.580082
36050    0.448531
36051    0.515618
36052    0.488265
36053    0.523377
36054    0.409296
36055    0.535402
36056    0.435592
36057    0.564455
36058    0.428961
36059    0.442477
36060    0.542688
36061    0.500820
36062    0.466423
36063    0.418483
36064    0.470097
36065    0.473675
36066    0

In [61]:
pred_csv = pd.concat([pred["t_id"],pd_ans[1]], axis=1)

In [63]:
pred_csv.head(10)

,t_id,1
0,35127,0.312819
1,5815,0.534440
2,21613,0.645487
3,24162,0.470157
4,13935,0.397954
5,6769,0.432818
6,23101,0.547083
7,27270,0.486919
8,2044,0.485804
9,10270,0.549066


In [65]:
pred_csv.to_csv("pred_file.csv")